In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

On s'interesse à la répartitions des emissions de gaz à effet de serre avec les données que l'on peut trouver sur le site climatewatcchdata https://www.climatewatchdata.org/data-explorer/historical-emissions?historical-emissions-data-sources=cait&historical-emissions-gases=all-ghg%2Cch4%2Cco2%2Cn2o%2Cf-gas&historical-emissions-regions=All%20Selected&historical-emissions-sectors=All%20Selected&page=1#data

In [2]:
#on lit les données
df_ges = pd.read_csv('CW_HistoricalEmissions_CAIT.csv')

In [31]:
df_ges.head()

,Country,Source,Sector,Gas,1990,1991,1992,1993,1994,1995,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,AFG,CAIT,Total excluding LUCF,All GHG,15.182846,15.102011,13.634687,13.463994,13.271733,13.476050,...,36.955463,44.906165,58.651861,66.749281,74.796108,84.619232,93.728621,95.372841,97.300113,98.920758
1,AFG,CAIT,Total including LUCF,All GHG,12.794043,12.713208,11.245884,11.075191,10.882930,11.087247,...,37.077364,45.028066,58.405642,66.503062,74.549889,84.373013,93.482402,95.527498,97.454770,99.075415
2,AFG,CAIT,Energy,All GHG,5.829497,5.334624,3.760858,3.422760,3.102594,2.783429,...,20.140819,26.056618,39.529423,47.592229,55.527034,64.675839,74.741644,75.932911,77.718177,79.580444
3,AFG,CAIT,Industrial Processes,All GHG,0.051879,0.054500,0.060111,0.062722,0.065343,0.067964,...,0.222971,0.248895,0.313896,0.378967,0.449909,0.534630,0.592081,0.758807,0.911544,1.064280
4,AFG,CAIT,Agriculture,All GHG,8.072853,8.396465,8.409491,8.486480,8.523959,8.957016,...,13.857023,15.788381,15.902260,15.777792,15.724861,16.220447,15.112568,15.315738,15.221951,14.744536


In [17]:
#pour d'avoir des contributions en doucles dans notre camembert on retire les colonnes de somme
sub_df_ges = df_ges.loc[(df_ges.Gas == 'All GHG') 
                        & (df_ges.Sector == 'Total excluding LUCF')
                        & (df_ges.Country != 'WORLD')]

In [262]:
#on peut tracer le camembert des emissions
fig = px.sunburst(sub_df_ges,path=['Country'], values ='1990' )
# fig.update_traces( textinfo='percent entry + label')
fig.show()

In [253]:
#On peut aussi le rendre interactif

df = px.data.tips()

figs = {
    y: px.sunburst(sub_df_ges, path=['Country'], values = y)
    for y in sub_df_ges.keys()[4:]
}

# choose a column that becomes the figure
fig = figs["1990"]
# now build menus, that use parameters that have been pre-built using plotly express
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": y,
                    "method": "restyle",
                    "args": [
                        {
                            "values": [figs[y].data[0].values],
                            "hovertemplate": figs[y].data[0].hovertemplate,
                        }
                    ],
                }
                for y in figs.keys()
            ]
        }
    ]
)

In [67]:
#pour filtrer un peu, on cherche les 10 pays les plus émetteurs en 2018
country_df = sub_df_ges.sort_values('2018', ascending = False).reset_index()
country = country_df.loc[:10,'Country']

In [241]:
print(country)

0     CHN
1     USA
2     EUU
3     IND
4     RUS
5     JPN
6     BRA
7     IDN
8     IRN
9     DEU
10    CAN
Name: Country, dtype: object


In [184]:
#on enlève les colones de sommes et on ne garde que les 10 plus gros emetteurs (2018)
sub_df_ges_2 = df_ges.loc[(~df_ges.Gas.isin(['All GHG'])) & 
                          (~df_ges.Sector.isin(['Total excluding LUCF','Total including LUCF','Bunker Fuels ', 'Fugitive Emissions '])) & 
                          (df_ges.Country.isin(country))
                        ]
                                            

In [245]:
#on s'assure qu'il n'y ait pas de valeur négative dans les colones
for keys in sub_df_ges_2.keys()[4:-1]:
    sub_df_ges_2.loc[sub_df_ges_2[keys] < 0, keys] = 0


In [225]:
sub_df_ges_2.loc[sub_df_ges_2['2018']<0,'2018']

Series([], Name: 2018, dtype: float64)

In [243]:
sub_df_ges_2.keys()[4:-1]

Index(['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018'],
      dtype='object')

In [248]:
fig2 = px.sunburst(sub_df_ges_2,path=['Sector','Gas','Country'], values ='2018' )
fig2.show()